In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.cloud import storage
from google.cloud.storage.bucket import Bucket

from initial import parse_company_page, post_process_company_df, parse_round_page, post_process_round_df, parse_investor_page, post_process_investor_df,\
    parse_acquisition_page, post_process_acquisition_df, map_company_status, map_acq_amount_to_num, CB_INVESTOR_TYPES_MAPPING, get_social_url, try_handle_date
import pandas as pd
from tqdm.notebook import tqdm
from utils import get_base_domain, cleanup_prefectur_info, get_usd, split2row_with_index, json_col_to_df, export_to_s3, validate_datetime_str_format, validate_date_str_format
from cb import generate_cb_uuid
from datetime import datetime
from loguru import logger


pd.options.display.max_columns = 999

In [3]:

client = storage.Client()
bucket: Bucket = client.get_bucket("initial-htmls")

# Parse data from GCS
## Parse Company Page

In [4]:
blobs = bucket.list_blobs(prefix="initial.inc/companies")
blob_cnt = sum(1 for _ in blobs)

company_df_list = []
blobs = bucket.list_blobs(prefix="initial.inc/companies")
for blob in tqdm(blobs, total=blob_cnt):
    html_byte = blob.download_as_string()
    df = parse_company_page(html_byte.decode('utf-8'))
    df['_timestamp'] = blob.name.split("/")[-1]
    company_df_list.append(df)

company_df = pd.concat(company_df_list, axis=0)
print(len(company_df))
company_df.head()

  0%|          | 0/409 [00:00<?, ?it/s]

20332


,Company name,Description,tag,INITIAL series,status,status update date,Assignee,memo,Representative's name,Total Funding Amount.(thousand yen),Total Procurement Calculation Date,Post-Money Valuation.(thousand yen),Valuation calculation round implementation date,Prefectures,Founded Date,Last Funding Date,type,registration date,update date,Investigation status,Market capitalization at the time of IPO (initial price).(One million yen),IPO date,IPO classification,number of employees,Shareholder status,origin,Universities and research institutes,Industry,market,address,Corporate number,phone number,website,_timestamp
0,"[{'name': 'Notus Solar Japan Co., Ltd.', 'url'...","[Using the solar sharing ""Notus Solar System"" ...",[],[],[],[],[],[],[Ryuzo Takahashi],[N/A],[],[],[],[Osaka],[2022/04/01],[2023/03/15],[private company],[2023/03/15],[2023/03/15],[],[],[],[],[],[],[],[],[Environment related],[],"[Room 1105, NLC Central Building, 4-3-21 Nishi...",[5120001245305],[],[https://notussolarjapan.co.jp/],1678867465018
1,"[{'name': 'Helte Co., Ltd.', 'url': '/companie...","[Providing “Sail”, a global communication serv...","[#edtech, #study, #online-service, #btoc]",[Series A],[],[],[],[],[Manabu Goto],"[226,548]",[2021/06/22],"[525,256]",[2021/05/19],[Chiba prefecture],[2016/03/22],[2023/03/01],[private company],[2018/11/02],[2023/03/15],[Under Investigation],[],[],[],[17],"[Uncertain about Investors, with Invested by c...",[],[],[Computers - IT Services],[],"[First Mitoya Building 3F, 2-28 Higashigamicho...",[2040001094614],[050-3358-6892],[https://www.helte.jp/],1678867465018
2,"[{'name': 'Posiwill Co., Ltd.', 'url': '/compa...",[Developing business centered on “POSIWILL CAR...,"[#human-development, #online-service, #career-...",[Series A],[],[],[],[],[Mei Kanai],"[219,844]",[2020/12/02],"[1,000,862]",[2020/08/03],[Tokyo],[2017/08/21],[2020/08/03],[private company],[2018/09/27],[2023/03/15],[Under Investigation],[],[],[],[42],[Invested by VCs],[],[],[Consumer services and sales],[],"[Tokyo Tatemono Aoyama Building 8F, 3-3-5 Kita...",[6010401133425],[],[https://www.posiwill.co.jp/],1678867465018
3,"[{'name': 'oVice Co., Ltd.', 'url': '/companie...",[Developed and provided a virtual space “oVice...,"[#organization, #hrtech, #cloud-services, #sof...",[Series B],[],[],[],[],[Ding Seho],"[6,049,286]",[2022/09/14],"[19,906,049]",[2022/08/26],[Ishikawa Prefecture],[2020/02/20],[2022/08/26],[private company],[2020/10/19],[2023/03/15],[Under Investigation],[],[],[],[125],[Invested by VCs],[],[],[business service],[],"[113 B, Honfuchu-cho, Nanao City, Ishikawa Pre...",[9010401151283],[],[https://ovice.in/],1678867465018
4,"[{'name': 'Things Inc.', 'url': '/companies/A-...",[Developed and operated cloud-based product de...,"[#cloud-services, #software, #btob, #hardware,...",[seed],[],[],[],[],[Atsuya Suzuki],"[259,006]",[2023/03/15],"[640,005]",[2023/02/07],[Tokyo],[2021/09/21],[2023/03/15],[private company],[2021/12/03],[2023/03/15],[Under Investigation],[],[],[],[],[Invested by VCs],[],[],[Computers - IT Services],[],"[5-25-18 Hongo, Bunkyo-ku, Tokyo Hi-Tech Hongo...",[1010001221329],[090-9392-1541],[https://things-inc.com/],1678867465018


In [5]:
company_df = post_process_company_df(company_df)
print(len(company_df))
company_df.head()

19529


,Company name,Description,tag,INITIAL series,status,status update date,Assignee,memo,Representative's name,Total Funding Amount.(thousand yen),Total Procurement Calculation Date,Post-Money Valuation.(thousand yen),Valuation calculation round implementation date,Prefectures,Founded Date,Last Funding Date,type,registration date,update date,Investigation status,Market capitalization at the time of IPO (initial price).(One million yen),IPO date,IPO classification,number of employees,Shareholder status,origin,Universities and research institutes,Industry,market,address,Corporate number,phone number,website,_timestamp,Company url
2,ALTURA X Inc.,Developed a system that realizes medical coope...,[],seed,,,,,Eito Sasakura,"123,210",2023/02/13,"[683,200]",2023/01/30,Osaka,2023/01/17,2023/01/30,private company,2023/02/02,2023/04/11,Under Investigation,,,,5,Invested by VCs,spin out,,,,"7-20-1 Fukushima, Fukushima-ku, Osaka-shi, Osa...",6120001251888,06-6123-8162,https://altura.co.jp/,1681438529879,/companies/A-45175
3,"MOVeLOT Co., Ltd.","Planning, development and operation of boardin...",[],,,,,,Kento Hiroi,"25,000",2023/03/09,[],,Tokyo,2023/02/01,2023/03/09,private company,2023/03/09,2023/04/11,Under Investigation,,,,,Invested by VCs,,,computer - hardware,,"Center of Garage, 1-16-3 Yokogawa, Sumida-ku, ...",3010601062617,,,1681438529879,/companies/A-45646
15,"Femlog Co., Ltd.","Development of “Femlog,” a period to EARN serv...",[],,,,,,Saori Nakai,"3,000",,[],,Tokyo,2023/01/26,2023/01/26,private company,2023/03/20,2023/03/21,Under Investigation,,,,,Uncertain about Investors,,,Consumer services and sales,,"Setagaya-ku, Tokyo",4010901052440,,,1681438529879,/companies/A-45849
14,"commissure Co., Ltd.",Utilizing the tactile presentation technology ...,[],,,,,,"Masateru Mizohashi, Arata Horie","2,000",,[],,Tokyo,2023/01/26,2023/01/26,private company,2023/03/22,2023/03/22,Under Investigation,,,,,Uncertain about Investors,Academic,University of Tokyo,Semiconductors/other electronic parts/products,,"Room 512, Collaborative Research Building, Kom...",4011001152305,,https://commissure.co.jp/,1681438529879,/companies/A-45880
13,"Egret・Lab Co., Ltd.",Technology development related to mass purific...,[],,,,,,Shinichi Yano,"4,000",,[],,Tokushima Prefecture,2023/02/06,2023/02/06,private company,2023/02/23,2023/03/23,Under Investigation,,,,,Uncertain about Investors,"Academic, from Local",National and Public Universities (Others),biotechnology,,"Tokushima City, Tokushima Prefecture",8480001012006,,,1681438529879,/companies/A-45419


## Parse funding round page

In [6]:
blobs = bucket.list_blobs(prefix="initial.inc/rounds")
blob_cnt = sum(1 for _ in blobs)

blobs = bucket.list_blobs(prefix="initial.inc/rounds")
rounds_df_list = []

for blob in tqdm(blobs, total=blob_cnt):
    html_byte = blob.download_as_string()
    df = parse_round_page(html_byte.decode('utf-8'))
    df['_timestamp'] = blob.name.split("/")[-1]
    rounds_df_list.append(df)

rounds_df = pd.concat(rounds_df_list, axis=0).reset_index(drop=True)
print(len(rounds_df))
rounds_df.head()

  0%|          | 0/1149 [00:00<?, ?it/s]

57395


,,Procurement date,Company Name,INITIAL series,Funding Amount.(thousand yen),"Pre-Money Valuation.(1,000 yen)",Post-Money Valuation .(thousand yen),Investors,_timestamp
0,[detail],[2023/04/01],"[{'name': 'Wellmo Co., Ltd.', 'url': '/compani...",[],[],[],[],[],1679117665404
1,[detail],[2023/03/24],"[{'name': 'Yura Holdings Co., Ltd.', 'url': '/...",[],"[16,704]",[],[],"[{'name': 'FUNDINNO', 'url': '/investors/V1023...",1679117665404
2,[detail],[2023/03/17],"[{'name': 'Editor Camp Co., Ltd.', 'url': '/co...",[],"[70,000]",[],[],"[{'name': 'East Ventures', 'url': '/investors/...",1679117665404
3,[detail],[2023/03/16],"[{'name': '24karat Inc.', 'url': '/companies/A...",[Seed],"[20,000]",[],[],[],1679117665404
4,[detail],[2023/03/16],"[{'name': 'AI Communis Pte.', 'url': '/compani...",[],[],[],[],"[{'name': 'Japanesepoint', 'url': '/investors/...",1679117665404


In [7]:
rounds_df = post_process_round_df(rounds_df)
print(len(rounds_df))
rounds_df.head()

57358


,Procurement date,Company Name,INITIAL series,Funding Amount.(thousand yen),"Pre-Money Valuation.(1,000 yen)",Post-Money Valuation .(thousand yen),Investors,_timestamp,Round Url,Company Url
57394,2023/03/01,"Creative Survey Co., Ltd.",Series B,"400,000","661,120","1,061,120",[],1681439004501,/companies/A-15659/funding_rounds/A-15659-08,/companies/A-15659
57366,2023/03/08,"RESTA Co., Ltd.",Seed,,,,"[{'name': 'RiskTaker Co., Ltd.', 'url': '/inve...",1681439004501,/companies/A-45992/funding_rounds/A-45992-02,/companies/A-45992
57373,2023/03/03,"Ecostyle Co., Ltd.",Series C,"1,873,200","12,333,015","14,206,215","[{'name': 'Mizuho Leasing CompanyLimited', 'ur...",1681439004501,/companies/A-30104/funding_rounds/A-30104-27,/companies/A-30104
57372,2023/03/03,"Zero Board Co., Ltd.",Series A,"461,213","10,467,537","10,928,750","[{'name': 'Nagase & Co., Ltd.', 'url': '/inves...",1681439004501,/companies/A-40337/funding_rounds/A-40337-23,/companies/A-40337
57371,2023/03/03,"Sky Blue Co., Ltd.",,"7,999",,,"[{'name': 'CommerceOneHoldings Inc.', 'url': '...",1681439004501,/companies/A-12907/funding_rounds/A-12907-26,/companies/A-12907


# Parse investor page

In [8]:
blobs = bucket.list_blobs(prefix="initial.inc/investors")
blob_cnt = sum(1 for _ in blobs)

blobs = bucket.list_blobs(prefix="initial.inc/investors")
investor_dicts = []

for blob in tqdm(blobs, total=blob_cnt):
    html_byte = blob.download_as_string()
    my_dict = parse_investor_page(html_byte.decode('utf-8'))
    investor_dicts.append({**my_dict, "Company url": '/' + '/'.join(blob.name.split("/")[-3:-1]), "_timestamp": blob.name.split("/")[-1]})

investor_df = pd.DataFrame(investor_dicts)
investor_df = investor_df[investor_df['Company url'] != '']
print(len(investor_df))
investor_df.head()

  0%|          | 0/8502 [00:00<?, ?it/s]

8502


,Company name,website,address,founded date,kinds,attributes,alias,snss,Company url,_timestamp
0,"Hokkaido Venture Capital, Inc.",http://www.hokkaido-vc.com/,"NCO Sapporo Station North Exit 2F, 2-20 Kitash...",August 1999,VCs,Independent,"HVC Co., Ltd. (former)",[],/investors/1,1679992252661
1,"Resona Capital Co., Ltd.",https://www.resona-gr.co.jp/resonacapital/,"Fukagawa Gatharia Tower S Building 16F, 1-5-25...",March 1988,VCs,"Banks, credit unions, credit unions",,[],/investors/10,1679992252661
2,ITOCHU Finance Corporation,http://www.itcfnc.co.jp/,"2-5-1 Kita-Aoyama, Minato-ku, Tokyo Itochu Bui...",November 1993,VCs,trading company,,[],/investors/1008,1679992252661
3,"TNP Partners, Corporation",http://www.tnp-g.jp,"Shin-Yokohama SR Building 8F, 3-6-1 Shin-Yokoh...",September 2000,VCs,Independent,"TSUNAMI Network Partners Co., Ltd. (former)",[],/investors/1013,1679992252661
4,"Knowledge Company Co., Ltd.",http://www.knowledgeco.jp/,,,,,,[],/investors/1015,1679992252661


In [9]:
investor_df = post_process_investor_df(investor_df)
investor_df.head()

,Company name,website,address,founded date,kinds,attributes,alias,snss,Company url,_timestamp
8464,ASIRO Inc.,https://asiro.co.jp/,"Shinjuku Island Wing 4F, 6-3-1 Nishi-Shinjuku,...",April 2016,business corporation,Others,,[],/investors/V19406,1681441468947
5694,DISCO Inc.,https://www.disc.co.jp/,東京都文京区後楽2-5-1 飯田橋ファーストビル 9階,October 1973,business corporation,Others,,[],/investors/V10341,1681441459138
8485,"Amusement Media Sougakuin Co., Ltd.",https://www.amgakuin.co.jp/,"2-29-8 Higashi, Shibuya-ku, Tokyo",December 1993,business corporation,Others,,[],/investors/V19531,1681441446883
7916,MCP Asset Management Co .Ltd.,https://www.mcp-am.jp/,"1-8-1 Otemachi, Chiyoda-ku, Tokyo KDDI Otemach...",,Financial institution,Other finance,,[],/investors/V17348,1681441434922
7678,GMO AD Marketing Inc.,https://www.gmo-am.jp/,"26-1 Sakuragaoka-cho, Shibuya-ku, Tokyo",September 1999,business corporation,IT-related,,[],/investors/V16560,1681441423184


# Parse aquisition page

In [10]:
blobs = bucket.list_blobs(prefix="initial.inc/finance_news")
blob_cnt = sum(1 for _ in blobs)

blobs = bucket.list_blobs(prefix="initial.inc/finance_news")
aquisition_dicts = []
html_type = ""

for blob in tqdm(blobs, total=blob_cnt):
    if blob.name == "initial.inc/finance_news/category=子会社化":
        continue
    html_byte = blob.download_as_string()
    my_dict = parse_acquisition_page(html_byte.decode('utf-8'))
    aquisition_dicts.append({**my_dict, "_timestamp": blob.name.split("/")[-1]})
    
acquisition_df = pd.DataFrame(aquisition_dicts)
acquisition_df = acquisition_df[~acquisition_df['acquirer'].isnull()]
print(len(acquisition_df))
acquisition_df.head()

  0%|          | 0/929 [00:00<?, ?it/s]

927


,Startup,Industry,Founded Date,Description,Date,Acquisition amount,acquirer,News URL,_timestamp,Procurement amount,Underwriter
0,"[{'name': 'ログミー株式会社', 'url': '/companies/A-158...",[消費者向けサービス・販売],[2013/08/08],[ネット上の動画を書き起こして読みやすくするWebメディア「ログミー」の展開。\n決算発表等...,[2020/08/31],[金額不明],"[{'name': 'Sansan株式会社', 'url': '/investors/V12...",[https://jp.corp-sansan.com/news/2020/ir_0826_...,1679822958827,NaN,NaN
1,"[{'name': 'Com Design Co., Ltd.', 'url': '/com...",[Computers - IT Services],[1997/01/01],"[Developed cloud type call center system ""CT-e...",[2020/09/30],[amount unknown],"[{'name': 'Tsuzuki Electric Co., Ltd.', 'url':...",[https://ssl4.eir-parts.net/doc/8157/tdnet/187...,1679822949868,NaN,NaN
2,"[{'name': '株式会社INMホールディングス', 'url': '/companie...",[ビジネスサービス],[2015/09/01],[インターネット上の共通ポイントサービスを創り上げた株式会社ネットマイルと独自仮想通貨「ビッ...,[2020/10/01],[金額不明],"[{'name': '株式会社ウィルズ', 'url': '/investors/V1485...",[https://biz.netmile.co.jp/news/press_2020/pre...,1679822941230,NaN,NaN
3,"[{'name': 'Plus Medi Co., Ltd.', 'url': '/comp...",[Medical/Healthcare ICT],[2016/12/15],"[Provision of the app ""MyHospital"" that suppor...",[2020/09/30],[amount unknown],"[{'name': 'Sumitomo Mitsui Financial Group, In...",[http://plus-medi-corp.com/2020/09/15/release/...,1679822920837,NaN,NaN
4,"[{'name': '株式会社マグネイチャー', 'url': '/companies/A-...",[産業・エネルギー],[2019/03/13],[ハルバッハモーター、ハルバッハ発電機をEV分野、電動航空機、有人ドローンなどの分野での普及...,[2020/11/06],[金額不明],"[{'name': '株式会社菊池製作所', 'url': '/investors/V074...",[https://ssl4.eir-parts.net/doc/3444/tdnet/188...,1679822904876,NaN,NaN


In [11]:
acquisition_df = split2row_with_index(acquisition_df, 'acquirer', show_index=True)
acquisition_df = post_process_acquisition_df(acquisition_df)
acquisition_df.head()

/Users/iansu/Dropbox/Mac (3)/Documents/Initial/notebooks/utils.py:166: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tmp = df[colname].apply(pd.Series, 1).stack().reset_index()


,Industry,Founded Date,Description,Date,Acquisition amount,News URL,_timestamp,Procurement amount,Underwriter,list_index,Startup Url,Startup Name,Acquirer Url,Acquirer Name
349,business service,2017/04/21,Operates the music studio reservation site “St...,2023/04/10,amount unknown,[https://prtimes.jp/main/html/rd/p/000000004.0...,1681439106655,NaN,NaN,0.0,/companies/A-30168,"Studiol Co., Ltd.",/investors/V19531,"Amusement Media Sougakuin Co., Ltd."
347,Consumer services and sales,2021/06/29,Development and operation of branding type NFT...,2023/04/04,amount unknown,[https://prtimes.jp/main/html/rd/p/000000015.0...,1681439103925,NaN,NaN,0.0,/companies/A-41462,"Memoria Co., Ltd.",/investors/V10007,"Book Live Co., Ltd."
345,business service,2020/09/18,"Together with local young creators, we produce...",2023/03/29,amount unknown,[https://prtimes.jp/main/html/rd/p/000000012.0...,1681439100351,NaN,NaN,0.0,/companies/A-42672,23 Inc.,/investors/V10341,Disco Inc.
344,business service,2016/09/12,Developing ICT services in the area of side jo...,2023/04/05,amount unknown,[https://prtimes.jp/main/html/rd/p/000000066.0...,1681439096649,NaN,NaN,0.0,/companies/A-28488,"Shumatsu Worker Co., Ltd.",/investors/V07121,CrowdWorks Inc.
342,computer - software,2007/09/10,"Provides ""Earthquake Information Real-time Twe...",2023/03/31,amount unknown,[https://prtimes.jp/main/html/rd/p/000000001.0...,1681439093639,NaN,NaN,0.0,/companies/A-10727,"Glucose Co., Ltd.",/investors/V08206,"PR TIMES Co., Ltd."


----------
# Transfer data to CB format
## Clean up company's prefecture table

In [12]:
company_df['Prefectures'] = cleanup_prefectur_info(company_df['Prefectures'])
company_prefectur_map = company_df.set_index('Company url')['Prefectures']

## Get CB Funding Round

In [13]:
fr_cols = ['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'country_code', 'state_code', 'region', 'city',
       'investment_type', 'announced_on', 'raised_amount_usd', 'raised_amount',
       'raised_amount_currency_code', 'post_money_valuation_usd',
       'post_money_valuation', 'post_money_valuation_currency_code',
       'investor_count', 'org_uuid', 'org_name', 'lead_investor_uuids']

for date_col in ['Procurement date']:
       rounds_df[date_col] = rounds_df[date_col].apply(lambda x: datetime.strptime(x, "%Y/%m/%d").strftime("%Y-%m-%d"))

rounds_df['uuid'] = rounds_df['Round Url'].apply(lambda x: generate_cb_uuid("funding_rounds", x))
rounds_df['name'] = rounds_df.apply(lambda row: f"{row['INITIAL series']} - {row['Company Name']}", axis=1)
rounds_df['type'] = 'funding_round'
rounds_df.rename(columns={'Round Url': 'permalink'}, inplace=True)
rounds_df['cb_url'] = ''
rounds_df['rank'] = ''
rounds_df['created_at'] = pd.to_datetime(rounds_df['_timestamp'], unit='ms').dt.strftime("%Y-%m-%d")
rounds_df['updated_at'] = pd.to_datetime(rounds_df['_timestamp'], unit='ms').dt.strftime("%Y-%m-%d")
rounds_df['country_code'] = 'JP'
rounds_df['state_code'] = ''
rounds_df['region'] = rounds_df['Company Url'].map(company_prefectur_map)
rounds_df['city'] = ''
rounds_df['investment_type'] = rounds_df['INITIAL series'].apply(lambda x: x.lower().replace(" ", "_"))
rounds_df['investment_type'] = rounds_df['investment_type'].apply(lambda x: x if x else 'undisclosed')
rounds_df.rename(columns={'Procurement date': 'announced_on'}, inplace=True)

rounds_df['raised_amount'] = rounds_df['Funding Amount.(thousand yen)'].apply((lambda x: float(x.replace(",", "")) * 1e3 if x else 0))
rounds_df['raised_amount_currency_code'] = 'JPY'
rounds_df['raised_amount_usd'] = rounds_df.apply(lambda row: get_usd(row['raised_amount_currency_code'], row['announced_on'], row['raised_amount']), axis=1)
rounds_df['post_money_valuation'] = rounds_df['Pre-Money Valuation.(1,000 yen)'].apply((lambda x: float(x.replace(",", "")) * 1e3 if x else 0))
rounds_df['post_money_valuation_currency_code'] = 'JPY'
rounds_df['post_money_valuation_usd'] = rounds_df.apply(lambda row: get_usd(row['post_money_valuation_currency_code'], row['announced_on'], row['post_money_valuation']), axis=1)

rounds_df['investor_count'] = rounds_df['Investors'].apply(len)
rounds_df['org_uuid'] = rounds_df['Company Url'].apply(lambda x: generate_cb_uuid("organization", x))
rounds_df.rename(columns={'Company Name': 'org_name'}, inplace=True)

rounds_df['lead_investor_uuids'] = ''

## Test Funding Round data

In [14]:
cb_rounds_df = rounds_df[fr_cols].copy()
assert (cb_rounds_df['created_at'] == '').sum() == 0, 'missing created_at value'
assert (cb_rounds_df['announced_on'] == '').sum() == 0, 'missing announced_on value'
assert cb_rounds_df['uuid'].nunique() == len(cb_rounds_df), 'It seems like the uuid is not unique across the table'
assert cb_rounds_df['investment_type'].isin(['undisclosed', 'seed', 'series_a', 'series_b', 'series_c', 'series_d', 'series_e', 'series_f', 'series_g']).all(), 'There are some invalid investment_type values'
for c in ['created_at', 'updated_at']:
    cb_rounds_df.loc[:, c] = cb_rounds_df[c].apply(lambda x: x + " 00:00:00")
    assert cb_rounds_df[c].apply(validate_datetime_str_format).all(), f'There are some invalid date format in cb_rounds_df[{c}]'
for c in ['announced_on']:
    assert cb_rounds_df[c].apply(validate_date_str_format).all(), f'There are some invalid date format in cb_rounds_df[{c}]'
for c in ['raised_amount', 'raised_amount_usd', 'post_money_valuation', 'post_money_valuation_usd']:
    cb_rounds_df[c] = cb_rounds_df[c].fillna(0).astype(int)
    assert cb_rounds_df[c].apply(lambda x: x >= 0).all(), f'There are some invalid value in cb_rounds_df[{c}]'

## Get investments data

In [15]:
investments_df = json_col_to_df(split2row_with_index(rounds_df[['Investors', 'name', 'permalink', 'uuid', 'announced_on', 'created_at', 'updated_at']], 'Investors', show_index=True).reset_index(drop=True), col_name='Investors', prefix=True)
investments_df = investments_df.drop_duplicates(['permalink', 'Investors_url', 'announced_on'])
investments_df['Investors_url'].fillna('', inplace=True)
investments_df['Investors_name'].fillna('', inplace=True)
investments_df = investments_df[~(investments_df['Investors_url'] == '')]

/Users/iansu/Dropbox/Mac (3)/Documents/Initial/notebooks/utils.py:166: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tmp = df[colname].apply(pd.Series, 1).stack().reset_index()


## **Get investors urls for crawling**

In [16]:
investors_to_crawl = investments_df.loc[~investments_df['Investors_url'].isin(investor_df['Company url']), ['Investors_name', 'Investors_url']].drop_duplicates()
investors_to_crawl.columns = ['name', 'url']
acquirers_to_crawl = acquisition_df.loc[~acquisition_df['Acquirer Url'].isin(investor_df['Company url']), ['Acquirer Name', 'Acquirer Url']].drop_duplicates()
acquirers_to_crawl.columns = ['name', 'url']
pd.concat([investors_to_crawl, acquirers_to_crawl], axis=0).drop_duplicates().to_csv('investors.csv', index=False)

In [17]:
inv_cols = ['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'funding_round_uuid', 'funding_round_name',
       'investor_uuid', 'investor_name', 'investor_type', 'is_lead_investor']


investments_df.rename(columns={'uuid': 'funding_round_uuid'}, inplace=True)
investments_df.rename(columns={'name': 'funding_round_name'}, inplace=True)
investments_df['uuid'] = investments_df.apply(lambda row: generate_cb_uuid("investments", row['permalink'] + row['Investors_url']), axis=1)

investments_df['name'] = investments_df.apply(lambda row: f"{row['Investors_name']} in {row['funding_round_name']}", axis=1)
investments_df['type'] = 'investment'
investments_df['permalink'] = ''
investments_df['cb_url'] = ''
investments_df['rank'] = ''
# investments_df['created_at'] = investments_df['created_at']
# investments_df['updated_at'] = investments_df['updated_at']
investments_df['investor_uuid'] = investments_df['Investors_url'].apply(lambda x: generate_cb_uuid("organization", x))
investments_df.rename(columns={'Investors_name': 'investor_name'}, inplace=True)
investments_df['investor_type'] = 'organization'
investments_df['is_lead_investor'] = False

cb_investments_df = investments_df[inv_cols].copy()

## Test Investment Data

In [18]:
assert investments_df['Investors_url'].isin(investor_df['Company url']).all()
assert (cb_investments_df['created_at'] == '').sum() == 0, 'missing created_at value'
assert cb_investments_df['uuid'].nunique() == len(cb_investments_df), 'It seems like the uuid is not unique across the table'
assert cb_investments_df['funding_round_uuid'].isin(cb_rounds_df['uuid']).all(), 'There are some invalid funding_round_uuid values'
for c in ['created_at', 'updated_at']:
    cb_investments_df.loc[:, c] = cb_investments_df[c].apply(lambda x: x + " 00:00:00").copy()
    assert cb_investments_df[c].apply(validate_datetime_str_format).all(), f'There are some invalid date format in cb_investments_df[{c}]'

## Get Investor Data

In [19]:
for date_col in ['founded date']:
       investor_df[date_col] = investor_df[date_col].apply(try_handle_date)

In [20]:
investor_cols = ['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at', 'updated_at', 
                 'roles', 'domain', 'country_code', 'state_code', 'region', 'city', 'investor_types', 'investment_count',
                 'total_funding_usd', 'total_funding', 'total_funding_currency_code', 'founded_on', 'closed_on',
                 'facebook_url', 'linkedin_url', 'twitter_url', 'logo_url']

investor_df['uuid'] = investor_df['Company url'].apply(lambda x: generate_cb_uuid("organization", x))
investor_df.rename(columns={'Company name': 'name'}, inplace=True)
investor_df['type'] = 'organization'
investor_df.rename(columns={'Company url': 'permalink'}, inplace=True)
investor_df['cb_url'] = ''
investor_df['rank'] = ''
investor_df['created_at'] = pd.to_datetime(investor_df['_timestamp'], unit='ms').dt.strftime("%Y-%m-%d")
investor_df['updated_at'] = pd.to_datetime(investor_df['_timestamp'], unit='ms').dt.strftime("%Y-%m-%d")
investor_df.loc[investor_df['created_at'] == '', 'created_at'] = investor_df.loc[investor_df['created_at'] == '', 'updated_at']
investor_df['roles'] = 'investor'
investor_df['domain'] = investor_df['website'].apply(get_base_domain).fillna('')
investor_df['country_code'] = 'JA'
investor_df['state_code'] = ''
investor_df['region'] = ''
investor_df['city'] = ''
investor_df['investor_types'] = investor_df['kinds'].map(CB_INVESTOR_TYPES_MAPPING)
investor_df['investment_count'] = investor_df['uuid'].map(investments_df.groupby(['investor_uuid']).size())
investor_df['total_funding_usd'] = 0
investor_df['total_funding'] = 0
investor_df['total_funding_currency_code'] = 'JPY'
investor_df.rename(columns={'founded date': 'founded_on'}, inplace=True)
investor_df['closed_on'] = ''
investor_df['facebook_url'] = investor_df['snss'].apply(lambda x: get_social_url(x, 'facebook'))
investor_df['linkedin_url'] = ''
investor_df['twitter_url'] = investor_df['snss'].apply(lambda x: get_social_url(x, 'twitter'))
investor_df['logo_url'] = ''

## Test Investor Data

In [21]:
cb_investor_df = investor_df[investor_cols].copy()
assert cb_investor_df['uuid'].nunique() == len(cb_investor_df), 'It seems like the uuid is not unique across the table'
assert (cb_investor_df['created_at'] == '').sum() == 0, 'missing created_at value'
assert (cb_investor_df['updated_at'] == '').sum() == 0, 'missing created_at value'


# Cleanup data without domain
cb_investor_df = cb_investor_df[~(cb_investor_df['domain']=='')].copy()
assert (cb_investor_df['domain']=='').sum() == 0, 'cb_investor_df has companies who missed domain value'
for c in ['created_at', 'updated_at']:
    cb_investor_df.loc[:, c] = cb_investor_df[c].apply(lambda x: x + " 00:00:00").copy()
    assert cb_investor_df[c].apply(validate_datetime_str_format).all(), f'There are some invalid date format in cb_organization_df[{c}]'

for c in ['founded_on', 'closed_on']:
    non_blank_cond = cb_investor_df[c] != ''
    cb_investor_df.loc[non_blank_cond, c] = cb_investor_df.loc[non_blank_cond, c].apply(lambda x: x + " 00:00:00").copy()
    assert cb_investor_df[c].apply(validate_datetime_str_format, blankable=True).all(), f'There are some invalid date format in cb_organization_df[{c}]'

for c in ['total_funding', 'total_funding_usd', 'investment_count']:
    cb_investor_df[c] = cb_investor_df[c].fillna(0).astype(int)
    assert cb_investor_df[c].apply(lambda x: x >= 0).all(), f'There are some invalid value in cb_organization_df[{c}]'

## Get Organization Data

In [22]:
for date_col in ['Total Procurement Calculation Date', 'Valuation calculation round implementation date', 'Founded Date', 'Last Funding Date', 'registration date', 'update date', 'IPO date']:
       company_df[date_col] = company_df[date_col].apply(lambda x: datetime.strptime(x, "%Y/%m/%d").strftime("%Y-%m-%d") if x else '')

In [23]:
# combine organization_df and investor_df
company_df['roles'] = 'company'
company_df['primary_role'] = 'company'
company_df['website'].fillna("", inplace=True)
print(f"Got total of {len(company_df)} companies, dropping {(company_df['website'] == '').sum()} companies without website")
company_df = company_df[company_df['website'] != '']

sub_investor_df = investor_df[['name', 'founded_on', '_timestamp', 'attributes', 'permalink', 'address', 'website']].copy()
sub_investor_df.columns = ['Company name', 'Founded Date', '_timestamp', 'Industry', 'Company url', 'address', 'website']
sub_investor_df['primary_role'] = 'investor'
print(f"Got total of {len(sub_investor_df)} investors, droping {(sub_investor_df['website'] == '').sum()} investors without website")
sub_investor_df = sub_investor_df[sub_investor_df['website'] != '']

organization_df = pd.concat([company_df, sub_investor_df], ignore_index=True, axis=0)
assert len(organization_df.columns) == len(company_df.columns), "concat company_df and sub_investor_df shouldn't increase the number of columns"
print(f"Total remaining organizations {len(organization_df)}")

Got total of 19529 companies, dropping 791 companies without website
Got total of 8502 investors, droping 2559 investors without website
Total remaining organizations 24681


In [24]:
org_cols = ['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'legal_name', 'roles', 'domain', 'homepage_url',
       'country_code', 'state_code', 'region', 'city', 'address',
       'postal_code', 'status', 'short_description', 'category_list',
       'category_groups_list', 'num_funding_rounds', 'total_funding_usd',
       'total_funding', 'total_funding_currency_code', 'founded_on',
       'last_funding_on', 'closed_on', 'employee_count', 'email', 'phone',
       'facebook_url', 'linkedin_url', 'twitter_url', 'logo_url', 'alias1',
       'alias2', 'alias3', 'primary_role', 'num_exits']

organization_df['uuid'] = organization_df['Company url'].apply(lambda x: generate_cb_uuid("organization", x))
organization_df.rename(columns={'Company name': 'name'}, inplace=True)
organization_df['type'] = "organization"
organization_df.rename(columns={'Company url': 'permalink'}, inplace=True)
organization_df['cb_url'] = ''
organization_df['rank'] = ''
organization_df['created_at'] = pd.to_datetime(organization_df['_timestamp'], unit='ms').dt.strftime("%Y-%m-%d")
organization_df['updated_at'] = pd.to_datetime(organization_df['_timestamp'], unit='ms').dt.strftime("%Y-%m-%d")

organization_df['legal_name'] = ''
# organization_df['roles'] Done before concat
organization_df['domain'] = organization_df['website'].apply(get_base_domain).fillna('')
organization_df.rename(columns={'website': 'homepage_url'}, inplace=True)
organization_df['country_code'] = "JP"
organization_df['state_code'] = ''
organization_df.rename(columns={'Prefectures': 'region'}, inplace=True)
organization_df['city'] = ''
# organization_df['address'] = organization_df['address']
organization_df['postal_code'] = ''
organization_df['status'] = organization_df['Investigation status'].map(map_company_status)

organization_df.rename(columns={'Description': 'short_description'}, inplace=True)
organization_df['category_list'] = [[]] * len(organization_df) # TODO. ask Conrad and Kalle to see if these twos are necessary, if necessary, generated by tags and industry
organization_df['category_groups_list'] = [[]] * len(organization_df)
organization_df['num_funding_rounds'] = organization_df['uuid'].map(cb_rounds_df.groupby('org_uuid').size())
organization_df['total_funding'] = organization_df['uuid'].map(cb_rounds_df.groupby('org_uuid')['raised_amount'].sum())
organization_df['total_funding_currency_code'] = 'JPY'
organization_df['total_funding_usd'] = organization_df['uuid'].map(cb_rounds_df.groupby('org_uuid')['raised_amount_usd'].sum())
organization_df.rename(columns={'Founded Date': 'founded_on'}, inplace=True)
organization_df['founded_on'].fillna('', inplace=True)
organization_df.rename(columns={'Last Funding Date': 'last_funding_on'}, inplace=True)
organization_df['last_funding_on'].fillna('', inplace=True)
organization_df['closed_on'] = ''
organization_df.rename(columns={'number of employees': 'employee_count'}, inplace=True)
organization_df['email'] = ''
organization_df.rename(columns={'phone number': 'phone'}, inplace=True)
organization_df['facebook_url'] = ''
organization_df['linkedin_url'] = ''
organization_df['twitter_url'] = ''
organization_df['logo_url'] = ''
organization_df['alias1'] = ''
organization_df['alias2'] = ''
organization_df['alias3'] = ''
# organization_df['primary_role'] Done before concat
organization_df['num_exits'] = ''

cb_organization_df = organization_df[org_cols].copy()

## Test Organization Data

In [25]:
assert cb_organization_df['uuid'].nunique() == len(cb_organization_df), 'It seems like the uuid is not unique across the table'
assert (cb_organization_df['created_at'] == '').sum() == 0, 'missing created_at value'

# Cleanup data without domain
assert (cb_organization_df['domain']=='').sum() == 0, 'cb_organization_df has companies who missed domain value'
for c in ['created_at', 'updated_at']:
    cb_organization_df.loc[:, c] = cb_organization_df[c].apply(lambda x: x + " 00:00:00").copy()
    assert cb_organization_df[c].apply(validate_datetime_str_format).all(), f'There are some invalid date format in cb_organization_df[{c}]'

for c in ['founded_on', 'last_funding_on', 'closed_on']:
    non_blank_cond = cb_organization_df[c] != ''
    cb_organization_df.loc[non_blank_cond, c] = cb_organization_df.loc[non_blank_cond, c].apply(lambda x: x + " 00:00:00").copy()
    assert cb_organization_df[c].apply(validate_datetime_str_format, blankable=True).all(), f'There are some invalid date format in cb_organization_df[{c}]'

for c in ['total_funding', 'total_funding_usd', 'num_funding_rounds']:
    cb_organization_df[c] = cb_organization_df[c].fillna(0).astype(int)
    assert cb_organization_df[c].apply(lambda x: x >= 0).all(), f'There are some invalid value in cb_organization_df[{c}]'


## Get IPO Data

In [26]:
ipo_cols = ['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'org_uuid', 'org_name', 'org_cb_url', 'country_code',
       'state_code', 'region', 'city', 'stock_exchange_symbol', 'stock_symbol',
       'went_public_on', 'share_price_usd', 'share_price',
       'share_price_currency_code', 'valuation_price_usd', 'valuation_price',
       'valuation_price_currency_code', 'money_raised_usd', 'money_raised',
       'money_raised_currency_code']

ipo_df = organization_df[organization_df['status'] == 'ipo'].copy()
ipo_df.rename(columns={'uuid': 'org_uuid'}, inplace=True)
ipo_df.rename(columns={'name': 'org_name'}, inplace=True)
ipo_df.rename(columns={'cb_url': 'org_cb_url'}, inplace=True)

ipo_df['uuid'] = ipo_df.apply(lambda row: generate_cb_uuid("ipo", row['permalink'] + row['IPO date']), axis=1)
ipo_df['name'] = ''
ipo_df['type'] = 'ipo'
ipo_df['permalink'] = ''
ipo_df['cb_url'] = ''
ipo_df['rank'] = ''
# created_at, updated_at, country_code, state_code, region, city already processed and exists

ipo_df['stock_exchange_symbol'] = ''
ipo_df['stock_symbol'] = ''
ipo_df.rename(columns={'IPO date': 'went_public_on'}, inplace=True)
ipo_df['share_price_usd'] = ''
ipo_df['share_price'] = ''
ipo_df['share_price_currency_code'] = 'JPY'
ipo_df['valuation_price'] = ipo_df['Market capitalization at the time of IPO (initial price).(One million yen)'].apply((lambda x: float(x.replace(",", "")) * 1e6 if x else 0))
ipo_df['valuation_price_currency_code'] = 'JPY'
ipo_df['valuation_price_usd'] = ipo_df.apply(lambda row: get_usd(row['valuation_price_currency_code'], row['went_public_on'], row['valuation_price']), axis=1)
ipo_df['money_raised_usd'] = ''
ipo_df['money_raised'] = ''
ipo_df['money_raised_currency_code'] = 'JPY'

## Test IPO Data

In [27]:
cb_ipo_df = ipo_df[ipo_cols].copy()

for c in ['created_at', 'updated_at']:
    cb_ipo_df.loc[:, c] = cb_ipo_df[c].apply(lambda x: x + " 00:00:00").copy()
    assert cb_ipo_df[c].apply(validate_datetime_str_format).all(), f'There are some invalid date format in cb_ipo_df[{c}]'

for c in ['went_public_on']:
    assert cb_ipo_df[c].apply(validate_date_str_format, blankable=True).all(), f'There are some invalid date format in cb_ipo_df[{c}]'
    
for c in ['valuation_price', 'valuation_price_usd']:
    cb_ipo_df[c] = cb_ipo_df[c].fillna(0).astype(int)
    assert cb_ipo_df[c].apply(lambda x: x >= 0).all(), f'There are some invalid value in cb_ipo_df[{c}]'

# Get Acquisition Data

In [28]:
acq_cols = ['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'acquiree_uuid', 'acquiree_name', 'acquiree_cb_url',
       'acquiree_country_code', 'acquiree_state_code', 'acquiree_region',
       'acquiree_city', 'acquirer_uuid', 'acquirer_name', 'acquirer_cb_url',
       'acquirer_country_code', 'acquirer_state_code', 'acquirer_region',
       'acquirer_city', 'acquisition_type', 'acquired_on', 'price_usd',
       'price', 'price_currency_code']

acquisition_df['Date'] = acquisition_df['Date'].apply(lambda x: datetime.strptime(x, "%Y/%m/%d").strftime("%Y-%m-%d") if x else '')

acquisition_df['uuid'] = acquisition_df.apply(lambda row: generate_cb_uuid("acquisition", row['Startup Url'] + row['Date'] + row['Acquirer Url']), axis=1)
acquisition_df['name'] = acquisition_df.apply(lambda row: f"{row['Startup Name']} acquired by {row['Acquirer Name']}", axis=1)
acquisition_df['type'] = 'acquisition'
acquisition_df['permalink'] = ''
acquisition_df['cb_url'] = ''
acquisition_df['rank'] = ''
acquisition_df['created_at'] = pd.to_datetime(acquisition_df['_timestamp'], unit='ms').dt.strftime("%Y-%m-%d")
acquisition_df['updated_at'] = pd.to_datetime(acquisition_df['_timestamp'], unit='ms').dt.strftime("%Y-%m-%d")
acquisition_df['acquiree_uuid'] = acquisition_df['Startup Url'].apply(lambda x: generate_cb_uuid("organization", x))
acquisition_df.rename(columns={'Startup Name': 'acquiree_name'}, inplace=True)
acquisition_df['acquiree_cb_url'] = ''
acquisition_df['acquiree_country_code'] = 'JP'
acquisition_df['acquiree_state_code'] = ''
acquisition_df['acquiree_region'] = acquisition_df['acquiree_uuid'].map(organization_df.set_index('uuid')['region'])
acquisition_df['acquiree_city'] = ''

acquisition_df['acquirer_uuid'] = acquisition_df['Acquirer Url'].apply(lambda x: generate_cb_uuid("organization", x))
acquisition_df.rename(columns={'Acquirer Name': 'acquirer_name'}, inplace=True)
acquisition_df['acquirer_cb_url'] = ''
acquisition_df['acquirer_country_code'] = 'JP'
acquisition_df['acquirer_state_code'] = ''
acquisition_df['acquirer_region'] = acquisition_df['acquirer_uuid'].map(organization_df.set_index('uuid')['region'])
acquisition_df['acquirer_city'] = ''

acquisition_df['acquisition_type'] = 'acquisition'
acquisition_df.rename(columns={'Date': 'acquired_on'}, inplace=True)

acquisition_df['price_currency_code'] = 'JPY'
acquisition_df['price'] = acquisition_df['Acquisition amount'].apply(map_acq_amount_to_num)
acquisition_df['price_usd'] = acquisition_df.apply(lambda row: get_usd(row['price_currency_code'], row['acquired_on'], row['price']), axis=1)

## Test acquisition data

In [29]:
cb_acquisition_df = acquisition_df[acq_cols].copy()

for c in ['created_at', 'updated_at']:
    cb_acquisition_df.loc[:, c] = cb_acquisition_df[c].apply(lambda x: x + " 00:00:00").copy()
    assert cb_acquisition_df[c].apply(validate_datetime_str_format).all(), f'There are some invalid date format in cb_acquisition_df[{c}]'
    
for c in ['price_usd', 'price']:
    cb_acquisition_df[c] = cb_acquisition_df[c].fillna(0).astype(int)
    assert cb_acquisition_df[c].apply(lambda x: x >= 0).all(), f'There are some invalid value in cb_acquisition_df[{c}]'

# Check data is properly joined

In [30]:
cb_investor_df.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,roles,domain,country_code,state_code,region,city,investor_types,investment_count,total_funding_usd,total_funding,total_funding_currency_code,founded_on,closed_on,facebook_url,linkedin_url,twitter_url,logo_url
8464,7827e727-6795-c7c0-6f99-8f4d49847c23,ASIRO Inc.,organization,/investors/V19406,,,2023-04-14 00:00:00,2023-04-14 00:00:00,investor,asiro.co.jp,JA,,,,corporate_venture_capital,0,0,0,JPY,2016-04-01 00:00:00,,,,,
5694,861a88df-de86-e5fc-cb9f-eed501e3819b,DISCO Inc.,organization,/investors/V10341,,,2023-04-14 00:00:00,2023-04-14 00:00:00,investor,disc.co.jp,JA,,,,corporate_venture_capital,0,0,0,JPY,1973-10-01 00:00:00,,,,,
8485,67401d8a-3f62-5aa7-3f43-20d83b809ca9,"Amusement Media Sougakuin Co., Ltd.",organization,/investors/V19531,,,2023-04-14 00:00:00,2023-04-14 00:00:00,investor,amgakuin.co.jp,JA,,,,corporate_venture_capital,0,0,0,JPY,1993-12-01 00:00:00,,,,,
7916,16b6ec3d-fffb-2f33-9525-56f70f47b7f6,MCP Asset Management Co .Ltd.,organization,/investors/V17348,,,2023-04-14 00:00:00,2023-04-14 00:00:00,investor,mcp-am.jp,JA,,,,private_equity_firm,1,0,0,JPY,,,,,,
7678,f69e2a03-0449-761f-e088-b891a3845dea,GMO AD Marketing Inc.,organization,/investors/V16560,,,2023-04-14 00:00:00,2023-04-14 00:00:00,investor,gmo-am.jp,JA,,,,corporate_venture_capital,1,0,0,JPY,1999-09-01 00:00:00,,,,,


In [31]:
org_uuids = cb_organization_df['uuid']
assert cb_organization_df.columns.tolist() == org_cols, "cb_organization_df columns are not the same as org_cols"

print("Remove rows in rounds_df where its org_uuid is not in org_uuids")
print(f"Remove {len(cb_rounds_df) - cb_rounds_df['org_uuid'].isin(org_uuids).sum()} rows of funding rounds data...")
cb_rounds_df = cb_rounds_df[cb_rounds_df['org_uuid'].isin(org_uuids)]
assert cb_rounds_df['org_uuid'].isin(org_uuids).all(), 'There are org_uuids in rounds_df that are not in org_uuids'
print(f"{len(cb_rounds_df)} rows of funding rounds data remain")
assert cb_rounds_df.columns.tolist() == fr_cols, "cb_rounds_df columns are not the same as fr_cols"

print("Remove rows in investments_df where its org_uuid is not in org_uuids")
print(f"Remove {len(cb_investments_df) - cb_investments_df['investor_uuid'].isin(org_uuids).sum()} rows of investments data...")
cb_investments_df = cb_investments_df[cb_investments_df['investor_uuid'].isin(org_uuids)]
assert cb_investments_df['investor_uuid'].isin(org_uuids).all(), 'There are investor_uuid in investments_df that are not in org_uuids'
print(f"{len(cb_investments_df)} rows of investment data remain")
assert cb_investments_df.columns.tolist() == inv_cols, "cb_investments_df columns are not the same as inv_cols"

assert cb_investor_df['uuid'].isin(org_uuids).all(), 'There are org_uuids in ipo_df that are not in org_uuids'
assert cb_investor_df.columns.tolist() == investor_cols, "cb_investor_df columns are not the same as inv_cols"

assert cb_ipo_df['org_uuid'].isin(org_uuids).all(), 'There are org_uuids in ipo_df that are not in org_uuids'
assert cb_ipo_df.columns.tolist() == ipo_cols, "cb_ipo_df columns are not the same as ipo_cols"

# Check acquiree_uuid
print("Remove rows in acquisition_df where its acquiree_uuid is not in org_uuids")
print(f"Remove {len(cb_acquisition_df) - cb_acquisition_df['acquiree_uuid'].isin(org_uuids).sum()} rows of acquisition data...")
cb_acquisition_df = cb_acquisition_df[cb_acquisition_df['acquiree_uuid'].isin(org_uuids)]
assert cb_acquisition_df['acquiree_uuid'].isin(org_uuids).all(), 'There are acquiree_uuid in cb_acquisition_df that are not in org_uuids'
print(f"{len(cb_acquisition_df)} rows of acquisition data remain")


# Check acquirer_uuid
print("Remove rows in acquisition_df where its acquirer_uuid is not in org_uuids")
print(f"Remove {len(cb_acquisition_df) - cb_acquisition_df['acquirer_uuid'].isin(org_uuids).sum()} rows of acquisition data...")
cb_acquisition_df = cb_acquisition_df[cb_acquisition_df['acquirer_uuid'].isin(org_uuids)]
assert cb_acquisition_df['acquirer_uuid'].isin(org_uuids).all(), 'There are acquirer_uuid in cb_acquisition_df that are not in org_uuids'
print(f"{len(cb_acquisition_df)} rows of acquisition data remain")
assert cb_acquisition_df.columns.tolist() == acq_cols, "cb_acquisition_df columns are not the same as acq_cols"

Remove rows in rounds_df where its org_uuid is not in org_uuids
Remove 2335 rows of funding rounds data...
55023 rows of funding rounds data remain
Remove rows in investments_df where its org_uuid is not in org_uuids
Remove 11123 rows of investments data...
35243 rows of investment data remain
Remove rows in acquisition_df where its acquiree_uuid is not in org_uuids
Remove 24 rows of acquisition data...
661 rows of acquisition data remain
Remove rows in acquisition_df where its acquirer_uuid is not in org_uuids
Remove 7 rows of acquisition data...
654 rows of acquisition data remain


# Log table size for traction

In [32]:
logger.add('initial_to_eva.log')
logger.info(f"cb_organization_df shape: {cb_organization_df.shape}")
logger.info(f"cb_rounds_df shape: {cb_rounds_df.shape}")
logger.info(f"cb_investments_df shape: {cb_investments_df.shape}")
logger.info(f"cb_investor_df shape: {cb_investor_df.shape}")
logger.info(f"cb_ipo_df shape: {cb_ipo_df.shape}")
logger.info(f"cb_acquisition_df shape: {cb_acquisition_df.shape}")

2023-04-22 23:37:19.266 | INFO     | __main__:<module>:2 - cb_organization_df shape: (24681, 41)
2023-04-22 23:37:19.268 | INFO     | __main__:<module>:3 - cb_rounds_df shape: (55023, 24)
2023-04-22 23:37:19.269 | INFO     | __main__:<module>:4 - cb_investments_df shape: (35243, 14)
2023-04-22 23:37:19.269 | INFO     | __main__:<module>:5 - cb_investor_df shape: (5943, 25)
2023-04-22 23:37:19.269 | INFO     | __main__:<module>:6 - cb_ipo_df shape: (520, 27)
2023-04-22 23:37:19.270 | INFO     | __main__:<module>:7 - cb_acquisition_df shape: (654, 27)


# Upload to S3

In [33]:
# export_to_s3...
export_to_s3(cb_organization_df, "organizations.csv")
export_to_s3(cb_rounds_df, "funding_rounds.csv")
export_to_s3(cb_investments_df, 'investments.csv')
export_to_s3(cb_ipo_df, 'ipos.csv')
export_to_s3(cb_acquisition_df, 'acquisitions.csv')
export_to_s3(cb_investor_df, 'investor.csv')